# Capstone: Airbnb Price Listing Prediction
## Part 4 Model Tuning

_Authors: Evonne Tham_

## 1. Import Necessary Libraries & Load Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# modelling
import time
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score, accuracy_score
# from xgboost import plot_importance


#Hide warnings
import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Load in Data 
train = pd.read_csv('../datasets/train.csv')

# #Set id as index 
# df.set_index('id', inplace=True)

print(f"Total Number of Listing: {train.shape[0]} | Total Number of Features: {train.shape[1]}")
train.head(4).T

---
## 2. Re-training the Best Model (XGBoost)

In [ ]:
# Create X and y variables
features = [col for col in train._get_numeric_data().columns if col != 'price']
X = train[features]
y = train['price']

# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, 
                                                  y, 
                                                  test_size=0.25,
                                                  random_state = 42) 

ss = StandardScaler()
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

In [ ]:
# Instantiate Best Model
xgb = XGBClassifier(gamma = 0, 
                    learning_rate = 0.05, 
                    max_depth = 5, 
                    n_estimators = 1000, 
                    reg_alpha = , 
                    reg_lambda = ,
                    subsample = 0.3)

In [ ]:
# Fit Model
xgb.fit(X_train, y_train)

--- 
## 3. Feature Selection

### 3.1. Feature Importances

In [ ]:
# # option 1 
# # ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['weight'], index=X_train.columns)
# # ft_weights_xgb_reg.sort_values('weight', ascending=False, inplace=True)
# # ft_weights_xgb_reg.head(10)

# # option 2
# #Visualizing top features in our production model. 
# features = xgb.feature_importances_
# key_features = pd.Series(features, index=X.columns)
# sorted_features = key_features.sort_values(ascending=False).head(30)
# sorted_features.head(10)

In [ ]:
#Visualizing top features in our production model. 
key_features = pd.DataFrame([xgb.feature_importances_], columns = X.columns).T
key_features.sort_values('weight', ascending = False, inplace = True)
key_features.head(10)

In [ ]:
# Plotting feature importances
plt.figure(figsize=(15,25))
plt.barh(key_features.index, key_features.weight, align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=20)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()

### 3.2. Dropping Features

## 4. Re-training the XGBoost with Selected Features